In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
!pip install pyspellchecker

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None


import seaborn as sns#Understanding my variables
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))    

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from spellchecker import SpellChecker

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

/kaggle/input/customer-support-on-twitter/sample.csv
/kaggle/input/customer-support-on-twitter/twcs/twcs.csv


In [8]:
full_data = pd.read_csv("../input/customer-support-on-twitter/twcs/twcs.csv")

This dataset will be very large, so I will be using a single companies customer service data.

In [9]:
pd.options.display.float_format = '{:,.0f}'.format
answers = full_data.loc[full_data['author_id'] == "sainsburys"]

In [10]:
full_data.shape

(2811774, 7)

In [11]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2811774 entries, 0 to 2811773
Data columns (total 7 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tweet_id                 int64  
 1   author_id                object 
 2   inbound                  bool   
 3   created_at               object 
 4   text                     object 
 5   response_tweet_id        object 
 6   in_response_to_tweet_id  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 131.4+ MB


In [12]:
answers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19466 entries, 773 to 2810390
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   tweet_id                 19466 non-null  int64  
 1   author_id                19466 non-null  object 
 2   inbound                  19466 non-null  bool   
 3   created_at               19466 non-null  object 
 4   text                     19466 non-null  object 
 5   response_tweet_id        8172 non-null   object 
 6   in_response_to_tweet_id  19417 non-null  float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 1.1+ MB


We reduced the dataset from 2811774 entries to 19466 entries, but we still need to include the sutomers tweets

In [13]:
answers.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
773,1324,sainsburys,False,Tue Oct 31 22:21:22 +0000 2017,"@115957 ...products. By 2020, we'll make sure ...",NaN,"1,325"
2404,3313,sainsburys,False,Tue Oct 31 22:51:55 +0000 2017,@116480 Oh no! That's never good Wendy. Did yo...,3314,"3,315"
2406,3316,sainsburys,False,Tue Oct 31 23:13:22 +0000 2017,@116480 I advise going into store and speaking...,3317,"3,314"
2549,3476,sainsburys,False,Wed Nov 22 10:01:03 +0000 2017,"@116534 Hi there, sorry about this. Could you ...",3477,"3,478"
2551,3479,sainsburys,False,Wed Nov 22 12:26:20 +0000 2017,"@116534 Thank you, can you confirm the right p...",NaN,"3,477"


We will get the customer tweets by using in_response_to_tweet to find what tweet the customer service of sainsburys was answering

In [14]:
answers.columns[answers.isnull().any()]

Index(['response_tweet_id', 'in_response_to_tweet_id'], dtype='object')

In [15]:
questionTweetId = []
answers["in_response_to_tweet_id"] = answers["in_response_to_tweet_id"].fillna(0.0).astype(int)

questionTweetId = answers["in_response_to_tweet_id"].to_list()


In [16]:
#Below was code used for various testing/troubleshooting


#tweetId = questionTweetId[3819]
#tweetId = 758160
#print(tweetId)

#index = full_data.index
#condition = full_data["tweet_id"] == tweetId
#indices = index[condition]
#indices_list = indices.tolist()

#condition1 = full_data["in_response_to_tweet_id"] == tweetId
#indices1 = index[condition1]
#indices_list1 = indices1.tolist()

#print(indices_list)

#print(indices_list1)
#if indices_list:
   # print(full_data.at[indices_list[0], "text"])
#if indices_list1:
    #print(full_data.at[indices_list1[0], "text"])


#full_data.iloc[[677615]]

This gets all the tweets from sainsburys and what they were responding to

In [17]:
QandA = []
i = len(questionTweetId)
for j in range(i):
    tweetId = questionTweetId[j]
    if tweetId != 0:
        index = full_data.index
        condition = full_data["tweet_id"] == tweetId
        indices = index[condition]
        indices_list = indices.tolist()
        condition1 = full_data["in_response_to_tweet_id"] == tweetId
        indices1 = index[condition1]
        indices_list1 = indices1.tolist()
        
        #Below was used for testing
        #print("TweetId:{}".format(tweetId))
        #print(indices_list[0])
        
        #The use of these if statements take advantage of the fact that empty sequences are false. This was implemented because of a missing tweetId, 758160
        if indices_list:
            Q = full_data.at[indices_list[0], "text"]   
        if indices_list1:
            A = full_data.at[indices_list1[0], "text"]
        QandA.append([Q, A])

Created a new dataset called data_clean that consists of 'Questions' and 'Answers'

In [18]:
data_clean = pd.DataFrame(QandA, columns =['Question', 'Answer'])
data_clean

,Question,Answer
0,@sainsburys what are you doing to reduce the u...,"@115957 ...products. By 2020, we'll make sure ..."
1,@sainsburys Nooo! I’ve just bought 2 sets of C...,@116480 Oh no! That's never good Wendy. Did yo...
2,@sainsburys I did thankfully! And this evening...,@116480 I advise going into store and speaking...
3,.@sainsburys what happened to the rest of the ...,"@116534 Hi there, sorry about this. Could you ..."
4,@sainsburys It’s a multi pack so I don’t have ...,"@116534 Thank you, can you confirm the right p..."
...,...,...
19412,which 1 of u sells banoffee pie @Tesco @sainsb...,"@822083 Oh, oh! We do! We do! You can see it h..."
19413,Dear @sainsburys - I bought a 2 litre bottle o...,@641812 Very sorry about that! I've contacted ...
19414,@sainsburys look how happy you’ve made this li...,"@620014 She looks lovely Sarah, I'm glad we co..."
19415,@sainsburys @122550 Why do you not state in th...,@822759 Hi there! Section 8 of our T&amp;C's s...


In [19]:
data_clean.shape

(19417, 2)

In [20]:
data_clean = data_clean.sample(n=2000)
data_clean

,Question,Answer
6493,@sainsburys - every time I order a @377245 yog...,@377244 ... using the below link please? Steve...
1792,@sainsburys @sainsburys ??,"@227322 Leave this with us, we'll have a look ..."
2379,Ah come on now @sainsburys. #everydaysexism ht...,@250442 So sorry about that. Which store did y...
4916,"@sainsburys That’s ok, it was an online order....",@337660 ...for you. Steven 2/2
7535,@sainsburys I had an item missing from my deli...,"@409585 Oh dear, sorry. Were you charged for t..."
...,...,...
13953,@sainsburys please thank #Farnham Water Lane S...,@635680 That's fantastic David. I've spoken to...
12787,"This may be a stupid question, but are your ca...","@586542 Hi there, that's not a stupid question..."
1870,@sainsburys Karen your treating this thing ver...,@230541 I'm terribly sorry Dick but Edna isn't...
10094,Dear @sainsburys - thanks for the freebee but ...,@498124 Sorry about that Hannah! Feel free to ...


In [21]:
data_clean["Original_Question"] = data_clean["Question"].astype(str)

Cleaning/pre-proccessing of the data

'Answer' has all @'s and Hashtags removed

In [22]:
data_clean["Original_Answer"] = data_clean["Answer"].astype(str)
data_clean["Answer"] = data_clean["Answer"].astype(str)
data_clean["Answer"] = data_clean.apply(lambda row: re.sub("\B#\w+", "", row['Answer']), axis=1)
data_clean["Answer"] = data_clean.apply(lambda row: re.sub("\B@\w+", "", row['Answer']), axis=1)

'cleanText' function created.
* Removes all @'s and hashtags
* Runs a spellchecker
* Removes stop words
* Removes puncutation
* Lemmatizes words

In [23]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
stopwords = set(stopwords.words('english'))
spell = SpellChecker()
def cleanText(text):
    text = str(text).lower()
    text = re.sub("\B#\w+", "", text)
    text = re.sub("\B@\w+", "", text)
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
        text = " ".join(corrected_text)
        text = " ".join([word for word in str(text).split() if word not in stopwords])
    text.translate(str.maketrans('', '', string.punctuation))
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])


applies 'cleanText' to 'Question'

In [24]:
data_clean["Question"] = data_clean["Question"].apply(cleanText)

In [25]:
data_clean

,Question,Answer,Original_Question,Original_Answer
6493,- every time order yoghurt send wrong flavour ...,... using the below link please? Steven 2/2 h...,@sainsburys - every time I order a @377245 yog...,@377244 ... using the below link please? Steve...
1792,,"Leave this with us, we'll have a look into th...",@sainsburys @sainsburys ??,"@227322 Leave this with us, we'll have a look ..."
2379,ah come . https://t.co/qbpymkn5pe,So sorry about that. Which store did you see ...,Ah come on now @sainsburys. #everydaysexism ht...,@250442 So sorry about that. Which store did y...
4916,that's ok online order email still smell nice,...for you. Steven 2/2,"@sainsburys That’s ok, it was an online order....",@337660 ...for you. Steven 2/2
7535,item miss delivery today,"Oh dear, sorry. Were you charged for this? Us...",@sainsburys I had an item missing from my deli...,"@409585 Oh dear, sorry. Were you charged for t..."
...,...,...,...,...
13953,please thank water lane store wife lizzie purs...,That's fantastic David. I've spoken to the st...,@sainsburys please thank #Farnham Water Lane S...,@635680 That's fantastic David. I've spoken to...
12787,may stupid question carve pumpkin edible,"Hi there, that's not a stupid question at all...","This may be a stupid question, but are your ca...","@586542 Hi there, that's not a stupid question..."
1870,karen treat thing lightly new york web page ed...,I'm terribly sorry Dick but Edna isn't employ...,@sainsburys Karen your treating this thing ver...,@230541 I'm terribly sorry Dick but Edna isn't...
10094,dear - thanks freebie i'm vegetarian https://t...,Sorry about that Hannah! Feel free to chuck i...,Dear @sainsburys - thanks for the freebee but ...,@498124 Sorry about that Hannah! Feel free to ...


function 'userAsk' created
* uses tfidf to get coseine similarity to all values in 'question'

In [28]:
def userAsk(text):
    tfidf=TfidfVectorizer()
    x_tfidf=tfidf.fit_transform(data_clean["Question"]).toarray()

    df_tfidf = pd.DataFrame(x_tfidf,columns=tfidf.get_feature_names())

    Question_tfidf = tfidf.transform([text]).toarray()

    cos=1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')#Find cosine similarity

    index_value = cos.argmax()
    
    #Resets the index of data_clean as otherwise it starts at like 6000 which means it won't work with index-value
    data_clean.reset_index(inplace = True, drop = True)

    return data_clean["Answer"].loc[index_value]

gets the new question and runs it through 'cleanText' & 'userAsk'

In [96]:
UserQuestion = input("Enter your Question: ")

Enter your Question:  I would like a refund! My chicken was raw!


In [97]:
UserQuestion = cleanText(UserQuestion)
UserAnswer = userAsk(UserQuestion)
print(UserAnswer)

 If you DM us on the below link your order number, email and postcode I'll arrange an evoucher to be emailed out to you. Thanks, Sam. https://t.co/MdMtnTlX6H
